
# AdventureWorksLT Data Warehouse


In [ ]:
# # install the Azure CLI
# # https://learn.microsoft.com/en-us/cli/azure/install-azure-cli-windows?tabs=azure-cli

# # add the Azure CLI extensions
# az extension add --name devops
# az extension add --name account
# az extension add --name databricks
# az extension add --name storage-preview

# # install the Databricks CLI
# pip install databricks-cli

# # install SQLCMD
# # https://learn.microsoft.com/en-us/sql/tools/sqlcmd/sqlcmd-utility?view=sql-server-ver16

# install dotnet SQl Project support 
# dotnet tool install -g microsoft.sqlpackage

In [ ]:
# check that all dependencies are installed

write-host "az CLI version" (az version | ConvertFrom-Json)."azure-cli"                                         # min 2.45
write-host "az CLI azure-devops version" (az extension show --name azure-devops | ConvertFrom-Json).version     # min  0.26
write-host "az CLI databricks version" (az extension show --name databricks | ConvertFrom-Json).version         # min 0.9
write-host "az CLI account version" (az extension show --name account | ConvertFrom-Json).version               # mim 0.2
write-host "databricks CLI version" $($null = (databricks --version) -match "[\d.]+"; $matches[0])              # min 0.17
write-host "sqlcmd version" $($null = (sqlcmd -? | where {$_ -like "Version*"}) -match "[\d.]+"; $matches[0])   # min 15.0
write-host "sqlpackage version" (SqlPackage /Version )                                                          # min 16.0

In [ ]:
# fill the following information

<#

# location to place the azure resources
$LOCATION = "eastus"

# random number
$SUFFIX   = "upid" + $(get-random) 

# Azure tenant and subscription ID
$env:TENANT_ID       = ""
$env:SUBSCRIPTION_ID = ""

# Azure DevOpv organization and personal access token
$env:AZURE_DEVOPS_ORG     = ""
$env:AZURE_DEVOPS_EXT_PAT = ""

# secrets for the SQL database
$env:LOCAL_IP           = (Invoke-WebRequest -uri "https://api.ipify.org/").Content
$env:SQL_ADMIN_USER     = "dbadmin"
$env:SQL_ADMIN_PASSWORD = ""

#>

. ./secrets.ps1

In [ ]:
# check the number of available vcores

$locationQuotas = az vm list-usage --location $LOCATION | ConvertFrom-Json
$selectedRegionVCores = [int] ($locationQuotas | where {$_.localName -eq "Total Regional vCPUs"}).limit

if(-not $selectedRegionVCores -gt 6) { throw "Not enough VCores" }

In [ ]:
# import databricks utility functions

. ./databricks/utilities.ps1
. ./azdevops/utilities.ps1

In [ ]:
# log in to azure

$null = az login 

# set the default subscription

az account set --subscription $env:SUBSCRIPTION_ID

## Creation of the OLTP environment


In [ ]:
$OLTP_RG      = "AdvWorksLT-OLTP-$SUFFIX"
$OLTP_SQL_SV  = "AdvWorksLTServer$SUFFIX"
$OLTP_SQL_DB  = "AdvWorksLTDatabase$SUFFIX"

In [ ]:
# create the OLTP resource group

$rgInfo = az group create `
    --location      $LOCATION `
    --name          $OLTP_RG

if( -not $? ) { $rgInfo; throw "Error" }

In [ ]:
# create the OLTP SQL server

$server = az sql server create `
    --name           $OLTP_SQL_SV `
    --resource-group $OLTP_RG `
    --location       $LOCATION `
    --admin-user     $env:SQL_ADMIN_USER `
    --admin-password $env:SQL_ADMIN_PASSWORD | 
    ConvertFrom-Json

if( -not $? ) { $server; throw "Error" }

# allow access to Azure services

$null = az sql server firewall-rule create `
    --resource-group   $OLTP_RG `
    --server           $OLTP_SQL_SV `
    --name             AllAzureIPsAccessRule `
    --start-ip-address 0.0.0.0 `
    --end-ip-address   0.0.0.0

# allow access to your IP

$null = az sql server firewall-rule create `
    --resource-group   $OLTP_RG `
    --server           $OLTP_SQL_SV `
    --name             LocalIPAccessRule `
    --start-ip-address $env:LOCAL_IP `
    --end-ip-address   $env:LOCAL_IP

In [ ]:
# create the OLTP SQL database

$database = az sql db create `
    --resource-group   $OLTP_RG `
    --server           $OLTP_SQL_SV `
    --name             $OLTP_SQL_DB `
    --compute-model    Serverless `
    --edition          GeneralPurpose `
    --family           Gen5 `
    --capacity         1 `
    --zone-redundant   false `
    --auto-pause-delay 60  `
    --backup-storage-redundancy local `
    --max-size         2GB | `
    ConvertFrom-Json

if( -not $? ) { $database; throw "Error" }

In [ ]:
# publish the OLTP SQL database from a bacpac

$output = SqlPackage /Action:Import `
    /TargetServerName:"$OLTP_SQL_SV.database.windows.net" `
    /TargetDatabaseName:$OLTP_SQL_DB `
    /TargetUser:$env:SQL_ADMIN_USER `
    /TargetPassword:$env:SQL_ADMIN_PASSWORD `
    /TargetEncryptConnection:False `
    /SourceFile:"./sql-database/bacpacs/AdvWorksLT.bacpac"

if (-not $?) { $output; throw "Error" }

## Creation of the OLAP environment

In [ ]:
$AzDevOpsProject    = "AdvWorksDW_$SUFFIX"

$OLAP_RG     = "AdvWorksLT-OLAP-$SUFFIX"
$OLAP_SQL_SV = "advworksdwsv$SUFFIX" 
$OLAP_SQL_DB = "advworksdw$SUFFIX"
$OLAP_DBS    = "databricksworkspace$SUFFIX"
$OLAP_SA     = "adls$SUFFIX"
$OLAP_KV     = "keyvault$SUFFIX"

In [ ]:
# set the azure devops defaults

az devops configure -d `
    organization=$env:AZURE_DEVOPS_ORG `
    project=$AzDevOpsProject

In [ ]:
# create the OLAP resource group

$rgInfo = az group create `
    --location      $LOCATION `
    --name          $OLAP_RG

if( -not $? ) { $rgInfo; throw "Error" }

In [ ]:
# creation of a service principal 

$spName = $OLAP_RG + "-Contributor"
$spInfo = az ad sp create-for-rbac --name $spName | ConvertFrom-Json

In [ ]:
# assign the contributor role at the rg level to the service principal

$role =  (az role definition list --name "Contributor" | ConvertFrom-Json).name
$scope = "/subscriptions/$env:SUBSCRIPTION_ID/resourcegroups/$OLAP_RG"

$raInfo = az role assignment create `
    --assignee $spInfo.appId `
    --role     $role `
    --scope    $scope

In [ ]:
# creation of a key vault and secrets

$keyVault = az keyvault create `
    --name           $OLAP_KV `
    --resource-group $OLAP_RG `
    --location       $LOCATION

if( -not $? ) { $keyVault; throw "Error" }

$secrets = @{

    "AdvWorksOLTPConnString" = "jdbc:sqlserver://$OLTP_SQL_SV.database.windows.net;databaseName=$OLTP_SQL_DB;user=$env:SQL_ADMIN_USER;password=$env:SQL_ADMIN_PASSWORD;"

    "AdvWorksOLAPServer"     = "$OLAP_SQL_SV.database.windows.net"
    "AdvWorksOLAPDatabase"   = $OLAP_SQL_DB 
    "AdvWorksOLAPUser"       = $env:SQL_ADMIN_USER
    "AdvWorksOLAPPassword"   = $env:SQL_ADMIN_PASSWORD
    "AdvWorksOLAPConnString" = "jdbc:sqlserver://$OLAP_SQL_SV.database.windows.net;databaseName=$OLAP_SQL_DB;user=$env:SQL_ADMIN_USER;password=$env:SQL_ADMIN_PASSWORD;"
}

foreach($_ in $secrets.GetEnumerator()) {
    $null = az keyvault secret set `
        --vault-name $OLAP_KV `
        --name       $_.Name `
        --value      $_.Value 
}

# grant get and list access to the service principal

$null = az keyvault set-policy `
    --name $OLAP_KV `
    --spn  $spInfo.appId `
    --secret-permissions get list

### Creation of the Az DevOps project

In [ ]:
# create an Azure DevOps project

$projectInfo = az devops project create `
    --name           $AzDevOpsProject `
    --source-control git `
    --visibility     private | 
    ConvertFrom-Json

### Creation of the Az DevOps Git repo

In [ ]:
# create a repo

$null = git add *
$null = git commit -m "initial commit [skip ci]"
git status

In [ ]:
# add a remote and push the changes

$repoURL = (az repos list | convertfrom-json)[0].remoteUrl
git remote add devopsrepo $repoURL
git push -u devopsrepo --all

### Creation of an Az ARM service connection

In [ ]:
# retrieve the subscription name

$subscription  = az account subscription list | 
    ConvertFrom-Json | 
    where {$_.subscriptionId -eq $env:SUBSCRIPTION_ID}

# create an Az Pipelines ARM Service Connection

$scDef = get-content ./azdevops/service-connections/azure-arm-sc-template.json | ConvertFrom-Json
$scDef.data.subscriptionId                          = $env:SUBSCRIPTION_ID
$scDef.data.subscriptionName                        = $subscription.displayName
$scDef.name                                         = "MyARMServiceConnection"
$scDef.authorization.parameters.tenantid            = $env:TENANT_ID
$scDef.authorization.parameters.serviceprincipalid  = $spInfo.appId
$scDef.authorization.parameters.serviceprincipalkey = $spInfo.password
$scDef.serviceEndpointProjectReferences[0].projectReference.id = $projectInfo.id
$scDef.serviceEndpointProjectReferences[0].projectReference.name = $AzDevOpsProject 
$scDef.serviceEndpointProjectReferences[0].name     = "MyARMServiceConnection"

$null = new-item -path ./tmp -ItemType directory
$scDef | convertto-json -depth 5 > ./tmp/azure-arm-sc-template.json

$scInfo = az devops service-endpoint create `
    --service-endpoint-configuration ./tmp/azure-arm-sc-template.json | 
    ConvertFrom-Json

# allow all pipelines to use the service connection

$null = az devops service-endpoint update `
    --id $scInfo.id --enable-for-all

### Creation of a variable group

In [ ]:
function Concat-PipelineArgs($hashtable) {
    $result = foreach ($elem in $hashtable.GetEnumerator() ){
        "$($elem.Name)=$($elem.Value)"
    }
    $result
}

In [ ]:
# creation of a variable group 

$VGName = "Variables"

$variables = @{
    RG_KEY_VAULT = $OLAP_KV
    SA_NAME      = $OLAP_SA
    TENANT_ID    = $env:TENANT_ID
}

# create a variable group and variables

$VGInfo = az pipelines variable-group create `
    --name      $VGName `
    --authorize false `
    --variables $(Concat-PipelineArgs $variables) | 
    ConvertFrom-json

# authorize access to all pipelines

$null = az pipelines variable-group update `
    --group-id  $VGInfo.id `
    --authorize true

### Creation of the Az Devops pipelines

In [ ]:
$pipelinesFolder = "./azdevops/pipelines"

foreach ($file in get-childitem $pipelinesFolder){

    $pipelineFile = $file.Name
    $pipelinePath = "$pipelinesFolder\$pipelineFile"
    $pipelineName = $file.Name.replace(".yml", "")
    
    $null = az pipelines create `
    --name        $pipelineName `
    --description "..." `
    --yml-path    $pipelinePath `
    --skip-first-run
}

### Creation of the OLAP SQL Database

In [ ]:
# create the OLAP SQL server

$server = az sql server create `
    --name           $OLAP_SQL_SV `
    --resource-group $OLAP_RG `
    --location       $LOCATION `
    --admin-user     $env:SQL_ADMIN_USER `
    --admin-password $env:SQL_ADMIN_PASSWORD | 
    ConvertFrom-Json

# allow access to Azure services

$null = az sql server firewall-rule create `
    --resource-group   $OLAP_RG `
    --server           $OLAP_SQL_SV `
    --name             AllAzureIPsAccessRule `
    --start-ip-address 0.0.0.0 `
    --end-ip-address   0.0.0.0

# allow access to your IP

$null = az sql server firewall-rule create `
    --resource-group   $OLAP_RG `
    --server           $OLAP_SQL_SV `
    --name             LocalIPAccessRule `
    --start-ip-address $env:LOCAL_IP `
    --end-ip-address   $env:LOCAL_IP

In [ ]:
# create the OLAP SQL database

$database = az sql db create `
    --resource-group   $OLAP_RG `
    --server           $OLAP_SQL_SV `
    --name             $OLAP_SQL_DB `
    --compute-model    Serverless `
    --edition          GeneralPurpose   `
    --family           Gen5 `
    --capacity         1 `
    --zone-redundant   false `
    --auto-pause-delay 60  `
    --backup-storage-redundancy local `
    --max-size         2GB  | `
    ConvertFrom-Json

In [ ]:
# deploy the sql project to the OLAP SQL database

$pipelineName = "olapsqldb-build"
$branch       = "refs/heads/main"

$pipelineRunInfo = az pipelines run `
    --branch     $branch `
    --name       $pipelineName |
    ConvertFrom-Json

write-host "Started pipeline with run id: " $pipelineRunInfo.id 
Wait-AzDevOpsPipelineTermination $pipelineRunInfo.id 

### Creation of the Databricks workspace

In [ ]:
# create a databricks workspace

$workspace = az databricks workspace create `
    --name           $OLAP_DBS `
    --resource-group $OLAP_RG `
    --location       $LOCATION `
    --require-infrastructure-encryption false `
    --sku            premium | ConvertFrom-Json

In [ ]:
# log in to the Databricks workspace

In [ ]:
# creation of a secret scope backed by a key vault
# go the following URL and enter the following values

$resourceID = "/subscriptions/$env:SUBSCRIPTION_ID/resourceGroups/$OLAP_RG/providers/Microsoft.KeyVault/vaults/$OLAP_KV"
$dnsName = "https://$OLAP_KV.vault.azure.net/"

"URL: https://$($workspace.workspaceUrl)#secrets/createScope"
"Scope Name: advworkslt"
"DNS Name: $dnsName"
"Resource ID: $resourceID"

In [ ]:
# generate Databricks PAT in the UI and save it into the variable env:DATABRICKS_TOKEN

# set the DATABRICKS_HOST and DATABRICKS_TOKEN env variables

$env:DATABRICKS_HOST  = "https://" + $workspace.workspaceUrl
$env:DATABRICKS_TOKEN = ""

In [ ]:
# test the connection to the databricks workspace

Assert-DatabricksConnection

In [ ]:
# create a databricks instance pool 

$poolInfo = databricks instance-pools create `
    --json-file ./databricks/instance-pools/testing-pool.json | 
    ConvertFrom-Json

In [ ]:
# create secrets for the Databricks host and PAT in the Key Vault

$secrets = @{
    "DATABRICKS-HOST" = $env:DATABRICKS_HOST
    "DATABRICKS-PAT"  = $env:DATABRICKS_TOKEN
}

foreach($_ in $secrets.GetEnumerator()) {
    $null = az keyvault secret set `
        --vault-name $OLAP_KV `
        --name       $_.Name `
        --value      $_.Value 
}

## Simulate Operations

### Build the advworksltdw library

In [ ]:
$pipelineName = "advworksdw-build"
$branch       = "refs/heads/main"

$pipelineRunInfo = az pipelines run `
    --branch     $branch `
    --name       $pipelineName |
    ConvertFrom-Json

write-host "Started pipeline with run id: " $pipelineRunInfo.id 
Wait-AzDevOpsPipelineTermination $pipelineRunInfo.id 

### Initial Load

In [ ]:
#  activate CDC on the source database

sqlcmd -I -b `
    -U $env:SQL_ADMIN_USER `
    -P $env:SQL_ADMIN_PASSWORD `
    -S "$OLTP_SQL_SV.database.windows.net" `
    -d $OLTP_SQL_DB `
    -i ./sql-database/scripts/cdc-setup.sql

In [ ]:
# run the initial load job

$runInfo = databricks jobs run-now `
    --job-id  $(Get-DatabricksJobId "AdvWorksDW_InitialLoad") `
    --version 2.1 | 
    ConvertFrom-Json 

Wait-DatabricksJobTermination $runInfo.run_id

In [ ]:
# open the AdventureWorksDW-SalesOrdersReport-SQLDatabase.pbit PBI template
# provide the following server and database info. Use SQL server authentication.

"$OLAP_SQL_SV.database.windows.net"
$OLAP_SQL_DB

### Delta Load for Day 1

In [ ]:
# run the sql database transaction for window 1

sqlcmd -I -b `
    -U $env:SQL_ADMIN_USER `
    -P $env:SQL_ADMIN_PASSWORD `
    -S "$OLTP_SQL_SV.database.windows.net" `
    -d $OLTP_SQL_DB `
    -i ./sql-database/scripts/trans/day_1.sql

if( -not $? ) { throw "Error" }

# sleep for 10 seconds

start-sleep -seconds 10

# trigger the advworksltdw_DeltaLoad job

$runInfo = databricks jobs run-now `
    --job-id  $(Get-DatabricksJobId "AdvWorksDW_DeltaLoad") `
    --version 2.1 | 
    ConvertFrom-Json 

Wait-DatabricksJobTermination $runInfo.run_id

In [ ]:
# refresh the PBI template to see the new data

### Delta Load for Day 2

In [ ]:
# run the sql database transaction for window 2

sqlcmd -I -b `
    -U $env:SQL_ADMIN_USER `
    -P $env:SQL_ADMIN_PASSWORD `
    -S "$OLTP_SQL_SV.database.windows.net" `
    -d $OLTP_SQL_DB `
    -i ./sql-database/scripts/trans/day_2.sql

if( -not $? ) { throw "Error" }

# sleep for 10 seconds

start-sleep -seconds 10

# trigger the advworksltdw_DeltaLoad job

$runInfo = databricks jobs run-now `
    --job-id  $(Get-DatabricksJobId "AdvWorksDW_DeltaLoad") `
    --version 2.1 | 
    ConvertFrom-Json 

Wait-DatabricksJobTermination $runInfo.run_id

In [ ]:
# refresh the PBI template to see the new data

## Clean up resources

In [ ]:
# delete the service principal

az ad sp delete --id $spInfo.appId

In [ ]:
# delete the devops project

az devops project delete --id $projectInfo.id --yes

In [ ]:
# delete the OLTP resource group

az group delete --name $OLTP_RG --yes

In [ ]:
# delete the OLAP resource group

az group delete --name $OLAP_RG --yes